In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3

from tqdm import tqdm
from datetime import datetime

tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

In [52]:
def create_nested_dataframe(df, filter_col, filter_val, header_col, header_val):
    """
    주어진 데이터프레임에서 특정 필터 조건에 맞는 데이터를 먼저 추출하고,
    다시 해당 조건 중 값을 가진 레코드를 헤더로 사용하는 중첩 데이터프레임을 생성

    Parameters:
    df (pd.DataFrame): 원본 데이터프레임
    filter_col (str): 필터링에 사용할 컬럼명
    filter_val (any): 필터링 조건에 사용할 값
    header_col (str): 헤더로 사용할 레코드를 찾기 위한 컬럼명
    header_val (any): 헤더로 사용할 레코드를 찾기 위한 값

    Returns:
    pd.DataFrame: 헤더를 설정한 새로운 중첩 데이터프레임
    """
    # 필터링 조건에 맞는 데이터프레임 생성
    filtered_df = df[df[filter_col] == filter_val]

    # 헤더로 사용할 레코드 찾기
    header = filtered_df[filtered_df[header_col] == header_val].iloc[0]

    # 나머지 데이터를 새로운 데이터프레임으로 구성
    nested_df = filtered_df[filtered_df[header_col] != header_val].reset_index(drop=True)

    # 새 데이터프레임의 컬럼명을 헤더로 설정
    nested_df.columns = header

    # 'Unnamed'로 시작하는 모든 열을 드롭
    nested_df = nested_df.loc[:, ~nested_df.columns.astype(str).str.startswith('Unnamed')]
    nested_df = nested_df.loc[:, ~nested_df.columns.astype(str).str.contains('nan', case=False)]

    return nested_df

In [11]:
## step 0 : define the path of the data

db_path = os.path.join(os.getcwd(), '2101_240526.sqlite3')
output_path = os.path.join(os.getcwd(), "2101_테스트용_빈 파일.xlsx")

In [4]:
## step 1 : connect to the database and read data as dataframe
conn = sqlite3.connect(db_path)
indiv_table_name = "2101_개별속성"
df = pd.read_sql_query(f'SELECT * FROM "{indiv_table_name}"', conn)

In [12]:
## step 2 : load working file
df_output = pd.read_excel(output_path)

In [48]:
df_output.head()

,SR No,작업자,출처,파일목록,공정,비고,Tag No,Tag No 수정,카테고리,클래스,...,속성3242,속성3243,속성3244,속성3245,속성3246,속성3247,속성3248,속성3249,속성3250,속성3251
0,SR No,작업자,출처,파일목록,공정,비고,Tag No,Tag No 수정,FIXED EQUIPMENT,VESSEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DOF522002061,AC,1.0PID,"B7S38-522-S0002,Column Data Sheet.xlsx,D-522-A...",522,NaN,522-C-108,522-C-108,FIXED EQUIPMENT,VESSEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DOF522004113,AC,1.0PID,"Column Data Sheet.xlsx,D-522-A205,HCR팀 - 522-L...",522,NaN,522-C-106,522-C-106,FIXED EQUIPMENT,VESSEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DOF522005601,AC,1.0PID,"Column Data Sheet.xlsx,D-522-A212,HCR팀 - 522-L...",522,NaN,522-C-109,522-C-109,FIXED EQUIPMENT,VESSEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SR No,작업자,출처,파일목록,공정,비고,Tag No,Tag No 수정,FIXED EQUIPMENT,VESSEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.head()

,SRNo,속성순번,속성값,공정,C|C|T,속성명,SR_No_ATTR
0,DOF522002061,속성1,3810mm /2740mm,522,FIXED EQUIPMENT|VESSEL|COLUMN,HEADER SIZE ID,DOF522002061|HEADER SIZE ID
1,DOF522004113,속성1,1680mm / 3048mm,522,FIXED EQUIPMENT|VESSEL|COLUMN,HEADER SIZE ID,DOF522004113|HEADER SIZE ID
2,DOF522005601,속성1,1220|mm,522,FIXED EQUIPMENT|VESSEL|COLUMN,HEADER SIZE ID,DOF522005601|HEADER SIZE ID
3,DOF522000246,속성1,1900|mm,522,FIXED EQUIPMENT|VESSEL|HORIZONTAL,HEADER SIZE ID,DOF522000246|HEADER SIZE ID
4,DOF522000344,속성1,2300|mm,522,FIXED EQUIPMENT|VESSEL|HORIZONTAL,HEADER SIZE ID,DOF522000344|HEADER SIZE ID


In [43]:
df_pivot = df[df['SRNo'] == "DOF522002061"]
df_pivot = df_pivot[['SRNo', '속성순번', '속성값']].pivot(index='SRNo', columns='속성명', values='속성값')
df_pivot = df_pivot.reset_index(drop=False)

In [56]:
nested_df = create_nested_dataframe(df_output, 'C|C|T', 'FIXED EQUIPMENT|VESSEL|COLUMN', '속성 그룹 코드', '01_속성명')

In [60]:
df_pivot

속성명,DESIGN CONDITION CORROSION ALLOWANCE HEAD,DESIGN CONDITION CORROSION ALLOWANCE SHELL,DESIGN CONDITION DESIGN PRESSURE CASE 1,DESIGN CONDITION DESIGN PRESSURE CASE 2,DESIGN CONDITION DESIGN TEMPERATURE CASE 1,DESIGN CONDITION DESIGN TEMPERATURE CASE 2,DESIGN CONDITION INSULATION THICKNESS,DESIGN CONDITION INSULATION TYPE,DESIGN CONDITION MAT HEAD,DESIGN CONDITION MAT SHELL,...,REFRACTORY TYPE,REMARK,SITE CONDITIONS AMB TEMPERATURE MAXIMUM,SITE CONDITIONS AMB TEMPERATURE MINIMUM,SITE CONDITIONS EARTHQUAKE FACTOR,SITE CONDITIONS WIND VELOCITY,WEIGHTS AND PAINT REQUIRED WEIGHT EMPTY,WEIGHTS AND PAINT REQUIRED WEIGHT FULL WATER,WEIGHTS AND PAINT REQUIRED WEIGHT OPERATING,WEIGHTS AND PAINT REQUIRED WEIGHT TEST
SRNo,,,,,,,,,,,,,,,,,,,,,
DOF522002061,CORROSION ALLOW. - TOP HEAD&SHELL TO 3M BELOW ...,CORROSION ALLOW. - TOP HEAD&SHELL TO 3M BELOW ...,3.5|kg/cm^2g,None,371|degC,None,70|mm,HOT,SA516-70 / SA516-70+410S S.S CLAD,SA516-70 / SA516-70+410S S.S CLAD,...,None,None,37.1|degC,-10.9|degC,ZONE 2A,45|m/s,142940|kg,473940|kg,194840|kg,None


In [61]:
nested_df

,작업자,출처,파일목록,공정,비고,Tag No,Tag No 수정,FIXED EQUIPMENT,VESSEL,COLUMN,...,DESIGN CONDITION MAT HEAD,DESIGN CONDITION TEST PRESSURE HYDROSTATIC,DESIGN CONDITION TEST PRESSURE PNEUMATIC,DESIGN CONDITION RADIO GRAPHY SHELL,DESIGN CONDITION RADIO GRAPHY HEAD,DESIGN CONDITION INSULATION TYPE,DESIGN CONDITION INSULATION THICKNESS,DESIGN CONDITION STRESS RELIEF,MATERIAL SHELL,MATERIAL HEAD
SR No,,,,,,,,,,,,,,,,,,,,,
DOF522002061,AC,1.0PID,"B7S38-522-S0002,Column Data Sheet.xlsx,D-522-A...",522,NaN,522-C-108,522-C-108,FIXED EQUIPMENT,VESSEL,COLUMN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOF522004113,AC,1.0PID,"Column Data Sheet.xlsx,D-522-A205,HCR팀 - 522-L...",522,NaN,522-C-106,522-C-106,FIXED EQUIPMENT,VESSEL,COLUMN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOF522005601,AC,1.0PID,"Column Data Sheet.xlsx,D-522-A212,HCR팀 - 522-L...",522,NaN,522-C-109,522-C-109,FIXED EQUIPMENT,VESSEL,COLUMN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
update_columns = df_pivot.columns.to_list()

In [59]:
nested_df.set_index('SR No', inplace=True)
df_pivot.set_index('SRNo', inplace=True)

In [64]:
nested_df.update(df_pivot[update_columns])

In [65]:
nested_df

,작업자,출처,파일목록,공정,비고,Tag No,Tag No 수정,FIXED EQUIPMENT,VESSEL,COLUMN,...,DESIGN CONDITION MAT HEAD,DESIGN CONDITION TEST PRESSURE HYDROSTATIC,DESIGN CONDITION TEST PRESSURE PNEUMATIC,DESIGN CONDITION RADIO GRAPHY SHELL,DESIGN CONDITION RADIO GRAPHY HEAD,DESIGN CONDITION INSULATION TYPE,DESIGN CONDITION INSULATION THICKNESS,DESIGN CONDITION STRESS RELIEF,MATERIAL SHELL,MATERIAL HEAD
SR No,,,,,,,,,,,,,,,,,,,,,
DOF522002061,AC,1.0PID,"B7S38-522-S0002,Column Data Sheet.xlsx,D-522-A...",522,NaN,522-C-108,522-C-108,FIXED EQUIPMENT,VESSEL,COLUMN,...,SA516-70 / SA516-70+410S S.S CLAD,5.03|kg/cm^2g,NaN,SPOT/FULL,FULL,HOT,70|mm,NaN,SA516-70 / SA516-70+410S S.S CLAD,SA516-70 / SA516-70+410S S.S CLAD
DOF522004113,AC,1.0PID,"Column Data Sheet.xlsx,D-522-A205,HCR팀 - 522-L...",522,NaN,522-C-106,522-C-106,FIXED EQUIPMENT,VESSEL,COLUMN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DOF522005601,AC,1.0PID,"Column Data Sheet.xlsx,D-522-A212,HCR팀 - 522-L...",522,NaN,522-C-109,522-C-109,FIXED EQUIPMENT,VESSEL,COLUMN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
